# Prospecção de Dados 2021/2022
## Second Home Assignment - Best Features and Model Selection
### Group 4
Students
- Catarina Canastra, nº57766
- Daniel Dias, nº59056
- João Raimundo, nº57454

#### 0.1 - Import Libraries

In [58]:
import time
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector, VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import explained_variance_score

# preprocessing and optimization
from sklearn.preprocessing import StandardScaler

# evaluation metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score

# config plot
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

random_state = 42

#### 0.2 - Import Dataset

In [59]:
dt = pd.read_csv("train.csv", sep=",")
df = pd.DataFrame(dt)
df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425000,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.250000,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237540,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,3572.596624,81.837500,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.905500,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440000,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.200000,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,3632.649185,90.890000,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.784400,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,775.425000,1010.820000,718.152900,939.009036,1.305967,0.773620,810.6,743.164286,323.811808,354.804183,160.250000,104.685714,136.126003,84.214573,1.259244,1.132547,205,49.314286,75.237540,67.797712,4654.35725,2999.159429,724.953211,53.974022,1.033129,0.760305,8958.571,1667.383429,3306.162897,3592.019281,81.837500,112.213571,60.123179,101.082152,1.159687,0.786007,127.05,81.207857,51.433712,41.639878,6.905500,3.822571,3.479475,1.037439,1.088575,0.927479,12.878,1.757143,4.599064,4.649635,107.756645,60.943760,7.062488,0.619095,0.308148,0.250477,399.97342,57.127669,168.854244,138.540613,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.

#### 0.3 - Exploratory Data Analysis

##### 0.3.1 - Profile Report

In [60]:
# generating profile report of the whole dataset
#profile = ProfileReport(df, title="PD_TPC2_Superconductors")
#profile.to_file("appendix/PD_TPC2_Superconductors.html")

##### 0.3.2 - Inspecting Data types and missing values

In [61]:
# data types
df.dtypes

number_of_elements         int64
mean_atomic_mass         float64
wtd_mean_atomic_mass     float64
gmean_atomic_mass        float64
wtd_gmean_atomic_mass    float64
                          ...   
range_Valence              int64
wtd_range_Valence        float64
std_Valence              float64
wtd_std_Valence          float64
critical_temp            float64
Length: 82, dtype: object

In [62]:
# missing values
df.isnull().sum()

number_of_elements       0
mean_atomic_mass         0
wtd_mean_atomic_mass     0
gmean_atomic_mass        0
wtd_gmean_atomic_mass    0
                        ..
range_Valence            0
wtd_range_Valence        0
std_Valence              0
wtd_std_Valence          0
critical_temp            0
Length: 82, dtype: int64

##### 0.3.3 - Cheking and dropping duplicates

In [63]:
# cheking for duplicates
Counter(df.duplicated())

Counter({False: 21197, True: 66})

In [64]:
# drop duplicates
df = df.drop_duplicates()

In [65]:
# cheking for duplicates
Counter(df.duplicated())

Counter({False: 21197})

Summary the dataset after deleting duplicates

In [66]:
# describe dataset
df.describe()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
count,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000,21197.000000
mean,4.120300,87.544201,72.945331,71.253164,58.474874,1.166782,1.065103,115.751945,33.204053,44.436401,41.490888,769.763322,870.845514,737.547500,833.083589,1.300496,0.927436,573.413988,483.935313,216.035193,224.476880,157.968942,134.637733,144.407310,120.880069,1.269041,1.132479,139.552012,51.290519,51.669299,52.410878,6104.949112,5259.357685,3448.167503,3104.482658,1.073272,0.856543,8673.258150,2900.587194,3418.704041,3321.751640,76.866669,92.749248,54.307039,72.413486,1.071182,0.771238,120.868857,59.383115,48.956611,44.445791,14.259618,13.798017,10.102742,10.097794,1.094502,0.915214,21.109508,8.183450,8.306107,7.698272,89.762371,81.597376,29.765853,27.233552,0.727696,0.539749,251.401535,62.118953,99.127605,96.419168,3.194431,3.148856,3.052542,3.051485,1.297020,1.053977,2.042553,1.480890,0.839692,0.674168,34.494532
std,1.438282,29.643136,33.460789,30.990774,36.615381,0.364712,0.401225,54.584784,26.966285,20.019732,19.973773,87.486110,143.208996,78.345068,119.722906,0.381597,0.334107,309.205105,224.068070,109.820525,127.811013,20.151776,28.777408,22.080187,35.801133,0.375109,0.406876,67.202224,34.993536,22.876341,25.271459,2841.463086,3216.978402,3697.026215,3970.072718,0.342333,0.319808,4093.005329,2398.872994,1671.437273,1609.618632,27.695062,32.282257,29.000902,31.666733,0.343299,0.286020,58.679836,28.598

##### 0.3.4 - Inferring Outliers

In [67]:
scalar = StandardScaler()
x = df.copy(deep=True)
column_names = []
for col in x.columns:
    column_names.append(col)

#Select numerical features
x = x[column_names].values

#Standardized values
x_standard = scalar.fit_transform(x)

standard_df = pd.DataFrame(x_standard, columns=column_names)

#Create dictionarie with the number of outliers for each feature
outliers = dict()
for i, val in enumerate(standard_df.columns):
    outliers[val] = dict()
    outliers[val]["count"] = len(np.where(abs(standard_df[val].values) > 3)[0])
    outliers[val]["propotion"] = outliers[val]["count"] / standard_df[val].shape[0]

outliers = pd.DataFrame(data=outliers).T
outliers.sort_values(by="count", ascending=False)


,count,propotion
std_FusionHeat,1115.0,0.052602
range_FusionHeat,1050.0,0.049535
wtd_std_FusionHeat,887.0,0.041846
range_ElectronAffinity,639.0,0.030146
wtd_mean_atomic_mass,537.0,0.025334
...,...,...
std_atomic_radius,0.0,0.000000
entropy_ThermalConductivity,0.0,0.000000
std_fie,0.0,0.000000
wtd_std_fie,0.0,0.000000


#### 0.4 - Data Partition

In [68]:
x_train = df.loc[:, df.columns != 'critical_temp']  # x_train set
x_train = pd.DataFrame(x_train, columns=x_train.columns)
x_train

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425000,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.250000,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237540,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,3572.596624,81.837500,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.905500,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440000,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.200000,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,3632.649185,90.890000,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.784400,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,775.425000,1010.820000,718.152900,939.009036,1.305967,0.773620,810.6,743.164286,323.811808,354.804183,160.250000,104.685714,136.126003,84.214573,1.259244,1.132547,205,49.314286,75.237540,67.797712,4654.35725,2999.159429,724.953211,53.974022,1.033129,0.760305,8958.571,1667.383429,3306.162897,3592.019281,81.837500,112.213571,60.123179,101.082152,1.159687,0.786007,127.05,81.207857,51.433712,41.639878,6.905500,3.822571,3.479475,1.037439,1.088575,0.927479,12.878,1.757143,4.599064,4.649635,107.756645,60.943760,7.062488,0.619095,0.308148,0.250477,399.97342,57.127669,168.854244,138.540613,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,775.4

In [69]:
y_train = df['critical_temp']  # y_train set
y_train = pd.DataFrame(y_train)
y_train

,critical_temp
0,29.00
1,26.00
2,19.00
3,22.00
4,23.00
...,...
21258,2.44
21259,122.10
21260,1.98
21261,1.84


In [70]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=random_state)

X_train = pd.DataFrame(X_train, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_test.columns)

Y_train = pd.DataFrame(Y_train)
Y_test = pd.DataFrame(Y_test)

print("Shapes:\n",
      "X_train: ", X_train.shape, "\n",
      "X_test: ", X_test.shape, "\n",
      "Y_train: ", Y_train.shape, "\n",
      "Y_test: ", Y_test.shape, "\n",
      )


Shapes:
 X_train:  (14201, 81) 
 X_test:  (6996, 81) 
 Y_train:  (14201, 1) 
 Y_test:  (6996, 1) 



#### 0.5 - Data Standardization

In [71]:
# Data Scaling 

scaler = StandardScaler()

x_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [72]:
x_train_scaled

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,-0.782197,1.355194,2.180013,1.442480,2.280814,-0.432528,-0.794273,0.229753,1.727512,0.397807,-0.403240,0.655526,-0.042104,1.114707,0.263273,-0.539547,-0.552544,-1.454721,0.241337,-1.438226,-1.588450,-0.426225,0.258402,0.144156,0.528668,-0.490985,-0.815531,-1.231549,0.677758,-1.235690,-1.031048,2.273734,1.950002,1.987333,1.616745,-0.370376,-0.330099,1.851766,1.672875,2.209485,2.626139,2.805334,2.597498,3.350898,3.242455,0.021738,-0.296389,-0.836299,2.117532,-0.902095,-1.154572,0.415185,0.449956,0.798697,0.737603,-0.096700,-0.396105,-0.405162,0.246150,-0.342602,-0.495874,2.452239,-0.405501,0.784876,-0.370198,-0.340902,0.295008,0.918076,-0.364187,1.076985,-0.072416,1.406508,2.279004,1.056995,2.328312,-0.748984,-0.963929,1.564077,2.381904,2.145551,0.057192
1,0.610304,0.057860,-0.627894,-0.022547,-0.636824,0.762305,0.810130,0.160989,-0.530300,0.118091,0.184703,-0.336711,0.921017,-0.577064,0.857495,0.617919,-0.063367,0.767899,0.964575,0.753193,0.937594,1.150132,-0.683358,0.526940,-0.869147,0.613529,0.707225,0.977086,-0.462184,1.102327,1.094862,-0.489582,-0.707665,-0.645007,-0.764769,0.731538,0.136613,0.061310,-0.350593,-0.247333,0.202321,-0.174367,0.517644,-0.045471,0.465773,0.831838,0.043179,0.106084,0.583908,0.054989,0.223364,-0.560185,-0.608495,-0.554588,-0.669810,0.762365,0.479294,-0.406257,-0.468284,-0.448315,-0.292162,-0.005093,0.321320,-0.626997,-0.650682,-0.826510,-1.052709,0.936842,0.696304,0.934583,1.102727,-0.571631,-0.890798,-0.538089,-0.836259,0.681403,0.580612,-0.042364,-0.479201,-0.089027,-0.743135
2,-1.478448,0.102857,0.639025,0.582157,0.944713,-1.346449,-1.141501,-1.546072,0.186206,-1.435362,-1.319149,1.198031,-0.096001,1.709973,0.178574,-1.606182,-0.754384,-1.394186,-1.492810,-1.318748,-1.218275,-0.816166,0.477226,-0.252185,0.696353,-1.592084,-1.316214,-1.261396,0.337617,-1.059031,-1.023575,0.970490,1.353939,1.310231,1.503158,-1.298140,-1.022768,-0.586283,1.006136,-0.168166,-0.177708,-0.345336,-0.879239,0.405905,-0.287533,-1.159797,-0.292677,-1.646492,-1.906840,-1.695309,-1.604452,0.712311,0.501392,1.133743,0.775186,-1.155892,-0.603892,-0.499700,-0.725896,-0.325474,-0.330597,-0.760112,-0.407714,0.876556,0.865147,-0.152609,0.208927,-1.454865,-0.845984,-1.476690,-1.357216,1.246980,1.024083,1.355200,1.101170,-1.558370,-0.982698,-0.845584,-0.879381,-0.706532,-0.426589
3,-0.085947,0.994871,0.880874,0.178165,-0.220028,-0.040494,-0.509000,1.193483,1.300057,1.232974,2

In [73]:
x_test_scaled

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,1.306555,0.334165,-0.419664,-0.014535,-0.626718,1.008033,1.083041,1.405015,-0.321791,1.227995,1.181915,-0.250489,0.992170,-0.320051,1.004719,1.146015,0.322742,0.617372,0.974192,0.417869,0.834902,0.196021,-1.042166,0.040596,-0.993816,1.174148,1.374300,0.514467,-0.830453,0.361047,0.524424,-0.258936,-0.704816,-0.580864,-0.766137,1.033259,1.162524,0.260752,-0.673723,0.374641,0.440543,-0.267787,0.434649,-0.645118,-0.039040,1.033708,0.268355,0.303600,0.614585,0.199037,0.341818,-0.411390,-0.619442,-0.429184,-0.671899,1.321201,1.682306,-0.222157,-0.586911,-0.369920,-0.313067,0.524079,-0.178020,-0.442188,-0.654768,0.738500,0.818370,0.936842,-0.226341,0.789459,0.650119,-0.507820,-0.624658,-0.536731,-0.668001,1.069458,0.967296,0.760856,-0.460712,0.539883,0.724916
1,0.610304,0.140465,-0.410864,0.016523,-0.603968,0.740366,0.234766,0.333730,0.001998,0.253318,0.751189,-0.219946,0.996772,-0.403260,0.919825,0.633091,-0.274733,0.646183,1.141963,0.683308,0.999207,0.383524,-0.709740,0.047412,-0.920298,0.646600,0.321668,0.753238,0.042812,0.645321,1.154730,-0.216270,-0.613330,-0.598906,-0.767364,0.831456,0.324563,0.061310,-0.481658,-0.195611,0.330969,0.085694,0.618124,0.451715,0.949392,1.130012,0.244078,-0.160853,0.747095,-0.244510,-0.181220,-0.576230,-0.661128,-0.569157,-0.685294,0.723624,0.505562,-0.406257,-0.559308,-0.435302,-0.360814,-0.041411,-0.455782,-0.650204,-0.662415,-0.971017,-0.848555,0.936842,-0.117210,0.945724,0.663216,-0.380198,-0.710579,-0.336916,-0.677101,0.704719,0.402872,-0.042364,-0.429805,-0.195379,-0.379847
2,-0.782197,0.252380,-0.414731,-0.262988,-0.753002,-0.994672,-0.866623,0.945576,0.244512,1.211634,1.480720,0.668856,1.900437,0.042562,2.200917,-0.804877,-1.227990,1.090642,1.957835,1.439668,0.440877,0.536182,-1.472135,-0.429557,-1.361974,-0.909165,-0.379473,1.156165,-0.493244,1.692161,0.775233,0.286790,-0.189206,-0.837170,-0.776760,-2.363929,-2.428749,2.564056,0.688110,3.160510,2.990152,0.483661,0.946174,0.949408,1.443471,-0.196763,-0.898839,-0.455934,1.419230,-0.453087,-0.658353,-0.154941,-0.365119,-0.745143,-0.708312,-2.232150,-2.061980,0.683414,0.011281,0.903393,0.991955,-0.357798,-0.844600,-0.686343,-0.670182,-0.488646,-0.968302,-0.514691,-0.500626,-0.476913,-0.384311,-0.188766,-0.205442,-0.731209,-0.456180,-1.145811,-0.772135,2.367297,-0.098634,2.710833,2.308579
3,-0.085947,-1.208293,-0.954077,-0.897135,-0.735068,0.204932,0.360330,-0.812840,-0.690344,-0.892239,-0.

#### 0.6 - Inferring the variance of attributes

In [74]:
c = X_train.columns
variances = []
for c in X_train:
    var = np.var(np.log(df[c]))
    #var = np.var(X_train[c])
    d = {"Features": c, "Variance": var}
    variances.append(d)

var_log = pd.DataFrame(variances)

fig = px.bar(var_log, x='Features', y='Variance', width=1000, height=800)
fig.update_layout(xaxis=dict(tickfont=dict(size=5)))
fig.update_layout(yaxis_title="Log-Variance")
fig.show()

C:\Users\raimu\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



#### 0.7 - Defining Functions

In [75]:
# function to test the explain variance score with linear regression, random forest regressor and decision tree regressor
def naif_model_testing(X_train, X_test, y_train, y_test):
    # test 3 approaches and print out the results

    rfr = RandomForestRegressor(n_estimators=100, random_state=random_state)
    rfr.fit(X_train, y_train)

    dtr = DecisionTreeRegressor(max_depth=5, random_state=random_state)
    dtr.fit(X_train, y_train)

    lmr = LinearRegression()
    lmr.fit(X_train, y_train)

    rf_preds = rfr.predict(X_test)
    dt_preds = dtr.predict(X_test)
    lr_preds = lmr.predict(X_test)

    print("RVE RFs: %7.4f" % explained_variance_score(y_test, rf_preds))
    print("RVE DTs: %7.4f" % explained_variance_score(y_test, dt_preds))
    print("RVE LRs: %7.4f" % explained_variance_score(y_test, lr_preds))

In [76]:
# function to select the best features with Select From Model algorithm for the regressors: Random Forest and Decision Tree
def sfm(x_train_scaled, Y_train, x_test_scaled, Y_test, estimator):
    N, M = x_train_scaled.shape

    est = estimator
    sel = SelectFromModel(estimator=est)
    sel.fit(x_train_scaled, Y_train.values.ravel())

    print("Threshold: ", sel.threshold_)

    features = sel.get_support()

    Features_selected = np.arange(M)[features]

    importances = sel.estimator_.feature_importances_
    features_index = sel.get_support(indices=True)

    importances_list = []
    features_name = []
    for i in features_index:
        importances_list.append(importances[i])
        features_name.append(x_train_scaled.columns[i])

    f_index_df = pd.DataFrame(Features_selected)
    fnames_df = pd.DataFrame(features_name)
    imp_df = pd.DataFrame(importances_list)

    imp_selected = pd.concat([f_index_df, fnames_df, imp_df], axis=1)
    imp_selected.columns = ["Index", "Selected Features", "Importance"]
    imp_selected_sort = imp_selected.sort_values(by="Importance", ascending=False)

    print("\nBest Selected Features with the respective Importance:\n")

    print(imp_selected_sort.to_string(index=False), "\n")

    print("\nExplained Variance Score:")
    nX_train = sel.transform(x_train_scaled)
    nX_test = sel.transform(x_test_scaled)
    naif_model_testing(nX_train, nX_test, Y_train.values.ravel(), Y_test.values.ravel())

In [77]:
# function to select the best features with Select From Model algorithm for Linear Regression classifier
def sfm_linear(x_train_scaled, Y_train, x_test_scaled, Y_test):
    N, M = x_train_scaled.shape

    est = LinearRegression()
    sel = SelectFromModel(estimator=est, threshold='1.25*mean')
    sel.fit(x_train_scaled, Y_train.values.ravel())

    print("Threshold: ", sel.threshold_)

    features = sel.get_support()

    Features_selected = np.arange(M)[features]

    coef = sel.estimator_.coef_
    features_index = sel.get_support(indices=True)

    coeficient_list = []
    features_name = []
    for i in features_index:
        coeficient_list.append(coef[i])
        features_name.append(x_train_scaled.columns[i])

    f_index_df = pd.DataFrame(Features_selected)
    fnames_df = pd.DataFrame(features_name)
    imp_df = pd.DataFrame(coeficient_list)

    imp_selected = pd.concat([f_index_df, fnames_df, imp_df], axis=1)
    imp_selected.columns = ["Index", "Selected Features", "Coeficient"]
    imp_selected_sort = imp_selected.sort_values(by="Coeficient", ascending=False)

    print("\nBest Selected Features with the respective Coeficient:\n")

    print(imp_selected_sort.to_string(index=False), "\n")

    print("\nExplained Variance Score:")
    nX_train = sel.transform(x_train_scaled)
    nX_test = sel.transform(x_test_scaled)
    naif_model_testing(nX_train, nX_test, Y_train.values.ravel(), Y_test.values.ravel())

In [78]:
# function to select the best features with Sequenctial Features Selector algorithm for Random Forest Regressor, Decision Tree Regressor and Linear Regression
def sfs(x_train_scaled, Y_train, x_test_scaled, Y_test, estimator):
    N, M = x_train_scaled.shape

    sfs = SequentialFeatureSelector(estimator, n_features_to_select=12)
    sfs.fit(x_train_scaled, Y_train.values.ravel())

    features = sfs.get_support()

    Features_selected = np.arange(M)[features]

    features_index = sfs.get_support(indices=True)

    features_name = []
    for i in features_index:
        features_name.append(x_train_scaled.columns[i])

    f_index_df = pd.DataFrame(Features_selected)
    fnames_df = pd.DataFrame(features_name)

    sfs_selected = pd.concat([f_index_df, fnames_df], axis=1)
    sfs_selected.columns = ["Index", "Selected Features"]

    print("\nBest Selected Features:\n")

    print(sfs_selected, "\n")

    print("\nExplained Variance Score:")
    nX_train = sfs.transform(x_train_scaled)
    nX_test = sfs.transform(x_test_scaled)
    naif_model_testing(nX_train, nX_test, Y_train.values.ravel(), Y_test.values.ravel())



In [79]:
# function to perform GridSearchCV
def best_model(x, y, estimator, params, cv):
    clf = GridSearchCV(estimator=estimator, param_grid=params, scoring='explained_variance', cv=cv, error_score='raise',
                       n_jobs=-1).fit(x, y)
    return {
        'Estimator': clf.best_estimator_,
        'Best Params': clf.best_params_,
        'Best Score': clf.best_score_,
        'Detailed Results': clf.cv_results_,
    }

In [80]:
# function to the prepare a dataframe with the final results of GridSearch
def result_to_file(algorithm_name, result_map, file_name):
    new_result = pd.DataFrame(result_map['Detailed Results'])
    new_result = new_result[['params', 'mean_test_score']]
    new_result = new_result.assign(Algorithm=algorithm_name)
    result = new_result[['Algorithm', 'params', 'mean_test_score']]
    result = pd.DataFrame(result)
    result.to_csv(file_name)
    return result

## 1- Selecting the Best Features and Dimensionality Reduction

In this step, we will find the best features with Select From Model and Sequential Features Selector algorithms support with a person correlation analysis as the first step.
Then, we will perform a PCA in order to reduce the dimensionality of the dataset.

#### 1.1 - Pearson Correlation

In [81]:
# correlation matrix
corr = df.corr(method='pearson')
fig = px.imshow(corr, text_auto=True, width=800, height=800)

fig.update_layout(
    yaxis=dict(tickfont=dict(size=5)),
    xaxis=dict(tickfont=dict(size=5)))

fig.show()

High degree: If the coefficient value lies between ± 0.50 and ± 1, then it is said to be a strong correlation. <br>
Moderate degree: If the value lies between ± 0.30 and ± 0.49, then it is said to be a medium correlation. <br>
Low degree: When the value lies below + . 29, then it is said to be a small correlation.

In [82]:
corr_target = pd.DataFrame(corr['critical_temp'])
corr_target = corr_target.drop(index="critical_temp")
corr_target.columns = ['critical_temp']

print("Features that are more correlated with target variable 'critical_temp':")
corr_target.loc[corr_target['critical_temp'] >= 0.5].sort_values(by='critical_temp', ascending=False)

Features that are more correlated with target variable 'critical_temp':


,critical_temp
wtd_std_ThermalConductivity,0.720960
range_ThermalConductivity,0.687282
range_atomic_radius,0.653394
std_ThermalConductivity,0.653232
wtd_entropy_atomic_mass,0.626536
wtd_entropy_atomic_radius,0.602986
number_of_elements,0.600394
range_fie,0.600179
wtd_std_atomic_radius,0.598972
entropy_Valence,0.598024


#### 1.2 - Select From Model

We will select the best features with Select From Model algorithm with Random Forest Regressor, Decision Tree Regressor and Linear Regression. Additionally, we evaluate the explained variance with RF, DT and LR in each step.

#### 1.2.1 - Random Forest Regressor

In [83]:
sfm(x_train_scaled, Y_train, x_test_scaled, Y_test, RandomForestRegressor(random_state=random_state))

Threshold:  0.012345679012345678

Best Selected Features with the respective Importance:

 Index               Selected Features  Importance
    67       range_ThermalConductivity    0.535776
    64   wtd_gmean_ThermalConductivity    0.122333
     9                 std_atomic_mass    0.029559
    74               wtd_gmean_Valence    0.022253
    66 wtd_entropy_ThermalConductivity    0.012598
    72                wtd_mean_Valence    0.012446 


Explained Variance Score:
RVE RFs:  0.9054
RVE DTs:  0.7628
RVE LRs:  0.5367


#### 1.2.2 - Decision Tree Regressor

In [84]:
sfm(x_train_scaled, Y_train, x_test_scaled, Y_test, DecisionTreeRegressor(random_state=random_state))

Threshold:  0.012345679012345678

Best Selected Features with the respective Importance:

 Index             Selected Features  Importance
    67     range_ThermalConductivity    0.537984
    64 wtd_gmean_ThermalConductivity    0.122310
    74             wtd_gmean_Valence    0.039756
     9               std_atomic_mass    0.032484
    78             wtd_range_Valence    0.014258
    47        range_ElectronAffinity    0.014253
    10           wtd_std_atomic_mass    0.013537
    31                  mean_Density    0.013443
    62  wtd_mean_ThermalConductivity    0.012556 


Explained Variance Score:
RVE RFs:  0.9104
RVE DTs:  0.7687
RVE LRs:  0.6062


#### 1.2.3 - Linear Regression

In [85]:
sfm_linear(x_train_scaled, Y_train, x_test_scaled, Y_test)

Threshold:  18.08131379095711

Best Selected Features with the respective Coeficient:

 Index            Selected Features  Coeficient
    22       wtd_mean_atomic_radius   86.176307
    72             wtd_mean_Valence   32.069839
    75              entropy_Valence   31.440442
    49         std_ElectronAffinity   28.530402
     1             mean_atomic_mass   27.228482
    14                wtd_gmean_fie   26.815100
     4        wtd_gmean_atomic_mass   25.051517
    62 wtd_mean_ThermalConductivity   23.832226
    25        entropy_atomic_radius   22.967635
    73                gmean_Valence   20.717748
    54         wtd_gmean_FusionHeat   19.613724
    17                    range_fie   19.120252
    26    wtd_entropy_atomic_radius   18.956435
    44   wtd_gmean_ElectronAffinity  -19.491360
    19                      std_fie  -20.478910
    47       range_ElectronAffinity  -23.460544
    76          wtd_entropy_Valence  -23.898121
    52          wtd_mean_FusionHeat  -26.328234
 

#### 1.3 - Sequential Feature Selector

We will select the best features with Sequential Feature Selector algorithm with Random Forest Regressor, Decision Tree Regressor and Linear Regression. Additionally, we evaluate the explained variance with RF, DT and LR in each step.

In [86]:
#sfs(x_train_scaled, Y_train, x_test_scaled, Y_test, RandomForestRegressor(random_state= random_state))

sfsRF_out = open("appendix/sfs_RandomForest.txt", "r")
print(sfsRF_out.read())

Best Selected Features:

    Index            Selected Features
0   3      gmean_atomic_mass          
1   9      std_atomic_mass            
2   10     wtd_std_atomic_mass        
3   15     entropy_fie                
4   17     range_fie                  
5   38     wtd_range_Density          
6   47     range_ElectronAffinity     
7   52     wtd_mean_FusionHeat        
8   61     mean_ThermalConductivity   
9   70     wtd_std_ThermalConductivity
10  71     mean_Valence               
11  72     wtd_mean_Valence            


Explained Variance Score:
RVE RFs:  0.9138
RVE DTs:  0.7340
RVE LRs:  0.5838


In [87]:
#sfs(x_train_scaled, Y_train, x_test_scaled, Y_test, DecisionTreeRegressor(random_state= random_state))

sfsDT_out = open("appendix/sfs_DecisionTree.txt", "r")
print(sfsDT_out.read())

Best Selected Features:

    Index             Selected Features
0   2      wtd_mean_atomic_mass        
1   11     mean_fie                    
2   15     entropy_fie                 
3   19     std_fie                     
4   37     range_Density               
5   38     wtd_range_Density           
6   46     wtd_entropy_ElectronAffinity
7   47     range_ElectronAffinity      
8   49     std_ElectronAffinity        
9   52     wtd_mean_FusionHeat         
10  77     range_Valence               
11  79     std_Valence                  


Explained Variance Score:
RVE RFs:  0.9120
RVE DTs:  0.6701
RVE LRs:  0.4933


In [88]:
#sfs(x_train_scaled, Y_train, x_test_scaled, Y_test, LinearRegression())

sfsLR_out = open("appendix/sfs_LinearRegression.txt", "r")
print(sfsLR_out.read())

Best Selected Features:

    Index                Selected Features
0   5      entropy_atomic_mass            
1   6      wtd_entropy_atomic_mass        
2   27     range_atomic_radius            
3   30     wtd_std_atomic_radius          
4   42     wtd_mean_ElectronAffinity      
5   44     wtd_gmean_ElectronAffinity     
6   50     wtd_std_ElectronAffinity       
7   60     wtd_std_FusionHeat             
8   66     wtd_entropy_ThermalConductivity
9   69     std_ThermalConductivity        
10  70     wtd_std_ThermalConductivity    
11  80     wtd_std_Valence                 


Explained Variance Score:
RVE RFs:  0.9141
RVE DTs:  0.7788
RVE LRs:  0.6655


#### 1.4 - Best Features Selected  

In [143]:
# best selected features by indice
selected_index = [15, 25, 44, 47, 49, 64, 67, 72, 74]

# column names of the best selected features
selected_names = list(x_test_scaled.columns[selected_index])

print("Best selected features by indice:", selected_index)
print("\nColumn names of the best selected features:\n", selected_names)

Best selected features by indice: [15, 25, 44, 47, 49, 64, 67, 72, 74]

Column names of the best selected features:
 ['entropy_fie', 'entropy_atomic_radius', 'wtd_gmean_ElectronAffinity', 'range_ElectronAffinity', 'std_ElectronAffinity', 'wtd_gmean_ThermalConductivity', 'range_ThermalConductivity', 'wtd_mean_Valence', 'wtd_gmean_Valence']


### 2 - Principal Components Analysis

In this step, we start by evaluate the explained total variance with 10 principal components:

In [89]:
# evaluating the explained total variance with 10 principal components
pca = PCA(n_components=10)
pca.fit(x_train_scaled)

tve = 0

for i, ve in enumerate(pca.explained_variance_ratio_):
    tve += ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve))
print()
print("Actual Eigenvalues:", pca.singular_values_, "\n")
for i, comp in enumerate(pca.components_):
    print("PC", i, "-->", comp, "\n")

PC0 - Variance explained:  0.3882 - Total Variance:  0.3882
PC1 - Variance explained:  0.1037 - Total Variance:  0.4919
PC2 - Variance explained:  0.0962 - Total Variance:  0.5881
PC3 - Variance explained:  0.0791 - Total Variance:  0.6672
PC4 - Variance explained:  0.0588 - Total Variance:  0.7260
PC5 - Variance explained:  0.0378 - Total Variance:  0.7637
PC6 - Variance explained:  0.0367 - Total Variance:  0.8004
PC7 - Variance explained:  0.0310 - Total Variance:  0.8315
PC8 - Variance explained:  0.0237 - Total Variance:  0.8552
PC9 - Variance explained:  0.0199 - Total Variance:  0.8751

Actual Eigenvalues: [668.26743995 345.36081648 332.64473513 301.55338751 260.02883809
 208.44864435 205.42801874 188.97348213 165.17890232 151.29299023] 

PC 0 --> [-0.15571443  0.05202323  0.09987778  0.08345308  0.1205011  -0.14650234
 -0.15640088 -0.12430574  0.08844361 -0.09850257 -0.11254083 -0.05576073
 -0.1285755  -0.02210194 -0.11690234 -0.15651251 -0.10941026 -0.16405123
 -0.08526872 -0.

In order to know how many principal components we will reduce our data into, we performed Scree plot and a Explained Variance by Components plot:

In [90]:
pc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.plot(pc, pca.singular_values_)
plt.title("Scree Plot", fontsize=14)
plt.xlabel('Principal component', fontsize=12)
plt.ylabel('Eigenvalues', fontsize=12)

Text(0, 0.5, 'Eigenvalues')

Analysing the Scree plot, the slope is steeper until the second component, so we must choose two principal components.

In [91]:
plt.figure(figsize=(8, 6))
plt.plot(pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
plt.title('Explained Variance by Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')

Text(0, 0.5, 'Cumulative Explained Variance')

However, the principal component must explain a comulative percentage above 70%. With the Explained Variance by Components plot, we can infer that 4 components explains 72.6% of the features variance. Only 58.8% was explained with 2 principal components.

<br>
Given the results we select 4 principal analysis components.

Next, we train the PCA model with 4 principal components and apply a transformation into the x_train_scaled and x_test_scaled:

In [92]:
pca = PCA(n_components=4)
pca.fit(x_train_scaled)

nX_train = pca.transform(x_train_scaled)
nX_test = pca.transform(x_test_scaled)
naif_model_testing(nX_train, nX_test, Y_train.values.ravel(), Y_test.values.ravel())

RVE RFs:  0.8652
RVE DTs:  0.6630
RVE LRs:  0.5048


In [93]:
xtrain_pc = pd.DataFrame(nX_train)
xtrain_pc.columns = ["PC1", "PC2", "PC3", "PC4"]
xtrain_pc

,PC1,PC2,PC3,PC4
0,5.414926,-2.041308,-2.633267,7.741479
1,-4.679600,-0.326168,2.389260,0.412023
2,7.903280,0.968208,0.413308,0.031601
3,2.493496,-2.655079,-6.081691,2.035512
4,-4.165676,0.817865,2.233788,0.452753
...,...,...,...,...
14196,5.951361,2.782436,4.341706,-6.968918
14197,5.586579,-6.029637,-5.473172,1.703294
14198,-5.030727,-0.618209,2.444568,0.144282
14199,-3.227114,1.178488,2.349165,2.204764


In [94]:
xtest_pc = pd.DataFrame(nX_test)
xtest_pc.columns = ["PC1", "PC2", "PC3", "PC4"]
xtest_pc

,PC1,PC2,PC3,PC4
0,-5.857828,-1.562840,-0.668722,-0.007667
1,-4.177553,-0.130448,1.924870,1.226300
2,-1.292051,4.083983,-0.614974,5.548708
3,-3.686521,2.174282,3.576733,-0.494505
4,-6.298011,-2.586264,-1.067430,-0.212216
...,...,...,...,...
6991,-4.067375,0.814397,2.607586,1.243962
6992,2.636201,-1.326014,-1.879953,-4.138144
6993,-5.405420,-1.481888,1.594204,-0.267734
6994,3.916529,-1.670146,-1.234818,-1.505546


### 3 - Finding the best model

After selecting the best features and reduce the dimensionality of train and test sets. We will find the best 10 models using the GridSearchCV algorithm (5 CV) with the Feature Selection Method set and the set of principal components individually, in order to conclude which method obtain the best results. The grid search will be performed with the following estimators: 

* Linear Regression 
* Decision Tree Regressor
* Random Forest Regressor
* Ada Boost with Decision Tree Regressor
* Ada Boost with Linear Regression
* Gradient Boosting Regressor
<br>

For the top 1 model obtained with the set of best features and principal components, we will test the model.

We start to perform a first run that will save the results in a csv, then we read and save the csv in a variable. All the results will be merged in a final results dataframe and consequently sorted by mean_test_score (R2 - coefficient of determination).

#### 3.1 - Feature Selection Method set

In [145]:
# finding the best model using feature selection method
X = x_train_scaled[selected_names]
Y = Y_train.values.ravel()



In [146]:
X

,entropy_fie,entropy_atomic_radius,wtd_gmean_ElectronAffinity,range_ElectronAffinity,std_ElectronAffinity,wtd_gmean_ThermalConductivity,range_ThermalConductivity,wtd_mean_Valence,wtd_gmean_Valence
0,-0.539547,-0.490985,3.242455,-0.836299,-0.902095,-0.370198,0.918076,2.279004,2.328312
1,0.617919,0.613529,0.465773,0.106084,0.054989,-0.650682,0.936842,-0.890798,-0.836259
2,-1.606182,-1.592084,-0.287533,-1.646492,-1.695309,0.865147,-1.454865,1.024083,1.101170
3,0.191103,0.174794,-0.309378,0.059178,0.001913,0.620059,-0.710165,0.989130,0.874163
4,0.654406,0.664978,0.760173,0.344025,0.409682,-0.662050,0.936842,-0.752523,-0.710930
...,...,...,...,...,...,...,...,...,...
14196,-1.616962,-1.555417,-1.921991,-1.336060,-1.275729,4.316540,-1.114070,-0.464910,-0.429188
14197,0.201414,0.307522,0.799781,0.214394,0.206587,1.565527,-0.981539,2.072669,2.168658
14198,0.621797,0.623916,0.452080,0.258230,0.274346,-0.645890,0.936842,-0.902438,-0.842973
14199,0.026302,-0.005710,0.992737,-0.508810,-0.381553,-0.662523,0.936842,-0.710579,-0.677101


In [147]:
Y

,critical_temp
19855,2.410
7240,75.000
12764,24.761
18455,1.290
9099,27.000
...,...
11285,1.700
11965,6.130
5390,60.000
860,15.000


In [148]:
# Linear Regression grid search - FIRST RUN ONLY

key = 'Linear Regression'
algorithm = LinearRegression()
params = [{}]
print(f'Now testing {key}')
linearResult = best_model(X, Y, algorithm, params, cv=5)
linearResult = result_to_file(key, linearResult, 'appendix/linear-result-fselection.csv')


Now testing Linear Regression


In [153]:
# Decision Tree Regressor grid search - FIRST RUN ONLY

key = 'Decision Tree Regressor'
algorithm = DecisionTreeRegressor()
params = [
    {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 2, 3, 4, 5],
        'min_samples_split': [2, 3, 4, 5, 6],
        'max_leaf_nodes': [None, 2, 3, 4, 5],
        'random_state': [random_state],
    }
]
print(f'Now testing {key}')
decisionTreeResult = best_model(X, Y, algorithm, params, cv=5)
decisionTreeResult = result_to_file(key, decisionTreeResult, 'appendix/decision-tree-result-fselection.csv')


Now testing Decision Tree Regressor


KeyError: 'squared_error'

In [151]:
# Random Forest Regressor grid search - FIRST RUN ONLY

key = 'Random Forest Regressor'
algorithm = RandomForestRegressor()
params = [
    {
        'n_estimators': [100, 500, 1000],
        'max_depth': [2, 3, 4, 5],
        'min_samples_split': [2, 3, 4],
        'max_leaf_nodes': [None, 2, 3, 4],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
random_forest_result = best_model(X, Y, algorithm, params, cv=5)
random_forest_result = result_to_file(key, random_forest_result, 'appendix/random-forest-result-fselection.csv')

Now testing Random Forest Regressor


KeyboardInterrupt: 

In [ ]:
# Ada Boost with Decision Tree Regressor grid search - FIRST RUN ONLY

key = 'Ada Boost with Decision Tree Regressor'
algorithm = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
params = [
    {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.2, 0.5, 0.8, 1],
        'loss': ['linear', 'square', 'exponential'],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
ada_boost_dt_result = best_model(X, Y, algorithm, params, cv=5)
ada_boost_dt_result = result_to_file(key, ada_boost_dt_result, 'appendix/ada-boost-dt-fselection.csv')

Now testing Ada Boost with Decision Tree Regression


In [ ]:
# Ada Boost with Linear Regression grid search - FIRST RUN ONLY

key = 'Ada Boost with Linear Regression'
algorithm = AdaBoostRegressor(base_estimator=LinearRegression())
params = [
    {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.2, 0.5, 0.8, 1],
        'loss': ['linear', 'square', 'exponential'],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
ada_boost_lr_result = best_model(X, Y, algorithm, params, cv=5)
ada_boost_lr_result = result_to_file(key, ada_boost_lr_result, 'appendix/ada-boost-lr-fselection.csv')

Now testing Ada Boost with Linear Regression


In [ ]:
# Gradient Boosting Regressor grid search - FIRST RUN ONLY

key = 'Gradient Boosting Regressor'
algorithm = GradientBoostingRegressor()
params = [
    {
        'n_estimators': [100, 500, 1000],
        'min_samples_split': [2, 3, 4, 5],
        'max_depth': [2, 3, 4, 5],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
gradient_boosting_result = best_model(X, Y, algorithm, params, cv=5)
gradient_boosting_result = result_to_file(key, gradient_boosting_result, 'appendix/gradient-boost-fselection.csv')

Now testing Gradient Boosting Regressor


#### 3.1.1 - Format the Final Results dataset

#### 3.2 - Principal Components set

In [96]:
# finding the best model using PCA method
X = xtrain_pc
Y = Y_train.values.ravel()

In [58]:
# Linear Regression grid search - FIRST RUN ONLY
'''
key = 'Linear Regression'
algorithm = LinearRegression()
params = [{}]
print(f'Now testing {key}')
linearResult = best_model(X, Y, algorithm, params, cv=5)
linearResult = result_to_file(key, linearResult, 'appendix/linear-result-PCA.csv')
'''

Now testing Linear Regression


In [60]:
# Decision Tree Regressor grid search - FIRST RUN ONLY
'''
key = 'Decision Tree Regressor'
algorithm = DecisionTreeRegressor()
params = [
    {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 2, 3, 4, 5],
        'min_samples_split': [2, 3, 4, 5, 6],
        'max_leaf_nodes': [None, 2, 3, 4, 5],
        'random_state': [random_state],
    }
]
print(f'Now testing {key}')
decisionTreeResult = best_model(X, Y, algorithm, params, cv=5)
decisionTreeResult = result_to_file(key, decisionTreeResult, 'appendix/decision-tree-result-PCA.csv')
'''

Now testing Decision Tree Regressor


In [107]:
# Random Forest Regressor grid search - FIRST RUN ONLY
'''
key = 'Random Forest Regressor'
algorithm = RandomForestRegressor()
params = [
    {
        'n_estimators': [100, 500, 1000],
        'max_depth': [2, 3, 4, 5],
        'min_samples_split': [2, 3, 4],
        'max_leaf_nodes': [None, 2, 3, 4],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
random_forest_result = best_model(X, Y, algorithm, params, cv=5)
random_forest_result = result_to_file(key, random_forest_result, 'appendix/random-forest-result-PCA.csv')
'''

Now testing Random Forest Regressor


In [63]:
# Ada Boost with Decision Tree Regressor grid search - FIRST RUN ONLY
'''
key = 'Ada Boost with Decision Tree Regressor'
algorithm = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
params = [
    {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.2, 0.5, 0.8, 1],
        'loss': ['linear', 'square', 'exponential'],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
ada_boost_dt_result = best_model(X, Y, algorithm, params, cv=5)
ada_boost_dt_result = result_to_file(key, ada_boost_dt_result, 'appendix/ada-boost-dt-PCA.csv')
'''

Now testing Ada Boost with Decision Tree Regression


In [64]:
# Ada Boost with Linear Regression grid search - FIRST RUN ONLY
'''
key = 'Ada Boost with Linear Regression'
algorithm = AdaBoostRegressor(base_estimator=LinearRegression())
params = [
    {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.2, 0.5, 0.8, 1],
        'loss': ['linear', 'square', 'exponential'],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
ada_boost_lr_result = best_model(X, Y, algorithm, params, cv=5)
ada_boost_lr_result = result_to_file(key, ada_boost_lr_result, 'appendix/ada-boost-lr-PCA.csv')
'''

Now testing Ada Boost with Linear Regression


In [108]:
# Gradient Boosting Regressor grid search - FIRST RUN ONLY
'''
key = 'Gradient Boosting Regressor'
algorithm = GradientBoostingRegressor()
params = [
    {
        'n_estimators': [100, 500, 1000],
        'min_samples_split': [2, 3, 4, 5],
        'max_depth': [2, 3, 4, 5],
        'random_state': [random_state]
    }
]
print(f'Now testing {key}')
gradient_boosting_result = best_model(X, Y, algorithm, params, cv=5)
gradient_boosting_result = result_to_file(key, gradient_boosting_result, 'appendix/gradient-boost-PCA.csv')
'''

Now testing Gradient Boosting Regressor


#### 3.2.1 - Format the Final Results dataset

In [135]:
# reading the result output file of each estimator, and tranform into dataframe
LR_result_pca = pd.DataFrame(pd.read_csv("appendix/linear-result-PCA.csv", sep=",")).drop(columns='Unnamed: 0')
DT_result_pca = pd.DataFrame(pd.read_csv("appendix/decision-tree-result-PCA.csv", sep=",")).drop(columns='Unnamed: 0')
RF_result_pca = pd.DataFrame(pd.read_csv("appendix/random-forest-result-PCA.csv", sep=",")).drop(columns='Unnamed: 0')
Ada_LR_result_pca = pd.DataFrame(pd.read_csv("appendix/ada-boost-lr-PCA.csv", sep=",")).drop(columns='Unnamed: 0')
Ada_DT_result_pca = pd.DataFrame(pd.read_csv("appendix/ada-boost-dt-PCA.csv", sep=",")).drop(columns='Unnamed: 0')
Gradient_result_pca = pd.DataFrame(pd.read_csv("appendix/gradient-boost-PCA.csv", sep=",")).drop(columns='Unnamed: 0')

# merging all in a final results dataframe
final_results_pca = LR_result_pca.append([LR_result_pca, DT_result_pca, RF_result_pca, Ada_LR_result_pca, Ada_DT_result_pca, Gradient_result_pca], ignore_index=True)

# sorting by mean_test_score - descending
final_results_pc_sorted = final_results_pca.sort_values(by="mean_test_score", ascending=False)

# selecting the top 10 models with the best mean_test_score
final_results_pc_top10 = final_results_pc_sorted.head(10)

print("\nTop 10 best models with feature selection:\n")
final_results_pc_top10 


Top 10 best models with feature selection:



,Algorithm,params,mean_test_score
1185,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.2, 'loss': 'square', 'n_estimators': 100, 'random_state': 42}",0.852275
1197,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.5, 'loss': 'exponential', 'n_estimators': 100, 'random_state': 42}",0.852186
1188,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.2, 'loss': 'exponential', 'n_estimators': 100, 'random_state': 42}",0.851817
1191,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.5, 'loss': 'linear', 'n_estimators': 100, 'random_state': 42}",0.850573
1183,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.2, 'loss': 'linear', 'n_estimators': 500, 'random_state': 42}",0.850042
1194,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.5, 'loss': 'square', 'n_estimators': 100, 'random_state': 42}",0.849272
1182,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.2, 'loss': 'linear', 'n_estimators': 100, 'random_state': 42}",0.848505
1206,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.8, 'loss': 'exponential', 'n_estimators': 100, 'random_state': 42}",0.847830
1189,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.2, 'loss': 'exponential', 'n_estimators': 500, 'random_state': 42}",0.846767
1200,Ada Boost with Decision Tree Regression,"{'learning_rate': 0.8, 'loss': 'linear', 'n_estimators': 100, 'random_state': 42}",0.843199


### 4 -  Testing the Best Model

#### 4.1 - Feature Selection Method set

#### 4.1 - Principal Components set